In [0]:
!pip install -q tensorflow-datasets

In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

W0817 18:25:01.741528 139991962249088 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
tf.enable_eager_execution()

In [4]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

W0817 18:25:02.142736 139991962249088 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [0]:
train_data, test_data = imdb["train"], imdb["test"]

In [0]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []


for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [0]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [8]:
training_labels_final

array([1, 1, 1, ..., 0, 1, 1])

In [9]:
training_sentences[1]

'b"This was soul-provoking! I am an Iranian, and living in th 21st century, I didn\'t know that such big tribes have been living in such conditions at the time of my grandfather!<br /><br />You see that today, or even in 1925, on one side of the world a lady or a baby could have everything served for him or her clean and on-demand, but here 80 years ago, people ventured their life to go to somewhere with more grass. It\'s really interesting that these Persians bear those difficulties to find pasture for their sheep, but they lose many the sheep on their way.<br /><br />I praise the Americans who accompanied this tribe, they were as tough as Bakhtiari people."'

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)

padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)


testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [12]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b this was soul provoking i am an iranian and living in <OOV> 21st century i didn't know that such big <OOV> have been living in such conditions at the time of my grandfather br br you see that today or even in <OOV> on one side of the world a lady or a baby could have everything served for him or her clean and on demand but here 80 years ago people <OOV> their life to go to somewhere with more grass it's really interesting that these <OOV> bear those difficulties to find <OOV> for their sheep but they lose many the sheep on their way br br i praise the americans who accompanied this tribe they were as
b"This was soul-provoking! I am an Iranian, and living in th 21st century, I didn't know that such big tribes have been living in such conditions at the time of my grandfather!<br /><br />You see that today, or even in 1925, on one side of the world a lady or a baby could have everything served for him or her clean and on-demand, but here 80 years ago, people ventured their life to go to

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 102       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [14]:
num_epochs = 10
model.fit(padded, 
          training_labels_final, 
          epochs=num_epochs, 
          validation_data=(testing_padded, testing_labels_final))

W0817 18:25:33.842377 139991962249088 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 8s 308us/sample - loss: 0.5479 - acc: 0.7578 - val_loss: 0.3827 - val_acc: 0.8417
Epoch 2/10
25000/25000 [==============================] - 8s 300us/sample - loss: 0.3300 - acc: 0.8642 - val_loss: 0.3333 - val_acc: 0.8562
Epoch 3/10
25000/25000 [==============================] - 7s 288us/sample - loss: 0.2688 - acc: 0.8915 - val_loss: 0.3328 - val_acc: 0.8558
Epoch 4/10
25000/25000 [==============================] - 7s 280us/sample - loss: 0.2325 - acc: 0.9100 - val_loss: 0.3485 - val_acc: 0.8510
Epoch 5/10
25000/25000 [==============================] - 7s 282us/sample - loss: 0.2068 - acc: 0.9216 - val_loss: 0.3702 - val_acc: 0.8457
Epoch 6/10
25000/25000 [==============================] - 7s 283us/sample - loss: 0.1865 - acc: 0.9325 - val_loss: 0.3972 - val_acc: 0.8408
Epoch 7/10
25000/25000 [==============================] - 7s 283us/sample - loss: 0.1687 - acc: 0.9411 - val_l

In [15]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
  
out_v.close()
out_m.close()

In [19]:
sentence = "I really think this is amazing. honest."

sequence = tokenizer.texts_to_sequences(sentence)

print(sequence)

[[11], [], [1430], [967], [4], [1537], [1537], [4737], [], [790], [2015], [11], [2924], [2191], [], [790], [2015], [11], [579], [], [11], [579], [], [4], [1783], [4], [4507], [11], [2924], [1277], [], [], [2015], [1005], [2924], [967], [579], [790], []]


In [0]:
try:
  from google.colab import files
  
except ImportError:
  pass

else:
  files.download('vecs.tsv')
  files.download('meta.tsv')